<a href="https://colab.research.google.com/github/miinsss/Capstone-Project/blob/main/Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libaries

In [41]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import recall_score
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.naive_bayes import ComplementNB, BernoulliNB
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.utils import resample

# Load the data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
# Load the contract_classification.csv file
contract_class_path = '/content/drive/MyDrive/Colab Notebooks/Capstone_Data/Capstone/Data/data_all_unique_values.csv'
contract_class = pd.read_csv(contract_class_path)



# Display the first few rows to verify content
print(contract_class.head())

<ipython-input-19-e723c9e46e08>:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  contract_class = pd.read_csv(contract_class_path)


   Unnamed: 0    ANO_SID CORPORATE_DEVISION           Bundesland    Typ  \
0           0  4114028.0                VHV  Nordrhein-Westfalen  Stadt   
1           1  4114039.0                VHV  Nordrhein-Westfalen  Stadt   
2           2  4114045.0                VHV  Nordrhein-Westfalen  Stadt   
3           3  4114049.0                VGV  Nordrhein-Westfalen  Stadt   
4           4  4114055.0                VHV  Nordrhein-Westfalen  Kreis   

   ORTPLZ CONSTRACTION_DESIGN  CONSTRUCTION_YEAR        WFL ZONE  ...  \
0   42109      NORMAL_VENTURE        1967.565648   69.00000  2.0  ...   
1   42277      NORMAL_VENTURE        1967.565648   65.00000  4.0  ...   
2   42389      DESIGN_CLASS_I        1967.565648   75.00000  1.0  ...   
3   42277      NORMAL_VENTURE        1967.565648  106.24368  NaN  ...   
4   42553      NORMAL_VENTURE        1967.565648  119.00000  2.0  ...   

   DRAIN_PIPE_INSURED  PRODUCTLINE PRIOR_DAMAGES  UVV-KZ  UNDERWRITER  YEAR  \
0                   0        So

In [20]:
print(contract_class['DAMAGE'].value_counts())


na_counts = contract_class.isna().sum()

print(na_counts)

DAMAGE
0    9330675
1     135686
Name: count, dtype: int64
Unnamed: 0                     0
ANO_SID                        0
CORPORATE_DEVISION             0
Bundesland                     0
Typ                            0
ORTPLZ                         0
CONSTRACTION_DESIGN            0
CONSTRUCTION_YEAR              0
WFL                            0
ZONE                      777969
TYPE_OF_DEDUCTIBLE             0
DRAIN_PIPE_INSURED             0
PRODUCTLINE                    0
PRIOR_DAMAGES                  0
UVV-KZ                         0
UNDERWRITER                    0
YEAR                           0
DAMAGE_HEAVY_RAIN_ZONE         0
LONGITUDE                      0
LATITUDE                       0
DAMAGE                         0
dtype: int64


# Models

# Naive Bayes

## Preprocessing

In [37]:
# Preprocessing from Francesca
# Clean data that is not needed for the model (or would be too hard to encode)
#contracts = contract_class.drop(['Unnamed: 0', 'SF-SYSTEM',  'ORTS-NAME', 'STRASSE', 'PARTY-ID', 'contract_year',
                          #  'Kreis', 'DAMAGE_FLOOD_ZONE', 'SUM_INSURED', 'PIPE_PREMIUM_AMOUNT', 'LONGITUDE',   ], axis=1)
contracts = contract_class.dropna()

# Convert all zone values to string (to reduce amount of unique values)
print('Original unique values:', contracts['ZONE'].nunique())
contracts['ZONE'] = contracts['ZONE'].astype('str')
print('New unique values:', contracts['ZONE'].nunique())

# Prepare binary variables
columns_to_encode = contracts.select_dtypes(include=['object']).columns
df_binary = pd.get_dummies(contracts, columns=columns_to_encode, drop_first=True)
# Replace negative values with zeros
df_binary[df_binary < 0] = 0
df_binary.info()

# Separate features and target variable
X = df_binary.drop(columns=['DAMAGE', 'ANO_SID' ])
y = df_binary['DAMAGE']

# Separate continuous and discrete features
continuous_features = ['CONSTRUCTION_YEAR','DAMAGE_HEAVY_RAIN_ZONE' ]
# The discrete features need to be adjusted after one-hot encoding
discrete_features = list(set(X.columns) - set(continuous_features))

# Ensure indices are aligned
contracts = contracts.reset_index(drop=True)
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)



Original unique values: 32


<ipython-input-37-c61c0ae77f8c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contracts['ZONE'] = contracts['ZONE'].astype('str')


New unique values: 23
<class 'pandas.core.frame.DataFrame'>
Index: 8688392 entries, 0 to 9466360
Data columns (total 72 columns):
 #   Column                                         Dtype  
---  ------                                         -----  
 0   Unnamed: 0                                     int64  
 1   ANO_SID                                        float64
 2   ORTPLZ                                         int64  
 3   CONSTRUCTION_YEAR                              float64
 4   WFL                                            float64
 5   TYPE_OF_DEDUCTIBLE                             int64  
 6   DRAIN_PIPE_INSURED                             int64  
 7   PRIOR_DAMAGES                                  int64  
 8   UVV-KZ                                         int64  
 9   YEAR                                           int64  
 10  DAMAGE_HEAVY_RAIN_ZONE                         float64
 11  LONGITUDE                                      float64
 12  LATITUDE                 

In [39]:
# Check for negative values
if (X_train_resampled[discrete_features] < 0).any().any():
    print("Negative values found in X_train_resampled for discrete features")
# Check for negative values in the training data for discrete features
negative_values = X_train_resampled[discrete_features] < 0
    # Extract and display the negative values
negative_values_df = X_train_resampled[discrete_features][negative_values]
print("Negative values in X_train_resampled:")
print(negative_values_df)

Negative values in X_train_resampled:
        ZONE_B ZONE_1.0 ZONE_2 Bundesland_Berlin  WFL  \
2941709    NaN      NaN    NaN               NaN  NaN   
5201100    NaN      NaN    NaN               NaN  NaN   
5592555    NaN      NaN    NaN               NaN  NaN   
4891623    NaN      NaN    NaN               NaN  NaN   
6033584    NaN      NaN    NaN               NaN  NaN   
...        ...      ...    ...               ...  ...   
3709730    NaN      NaN    NaN               NaN  NaN   
5940216    NaN      NaN    NaN               NaN  NaN   
2579374    NaN      NaN    NaN               NaN  NaN   
3551931    NaN      NaN    NaN               NaN  NaN   
3483251    NaN      NaN    NaN               NaN  NaN   

        CONSTRACTION_DESIGN_DESIGN_CLASS_III Typ_Stadt ZONE_7.0  \
2941709                                  NaN       NaN      NaN   
5201100                                  NaN       NaN      NaN   
5592555                                  NaN       NaN      NaN   
4891623  

In [36]:
# Assuming you have a DataFrame named 'contracts'
unique_values = df_binary['PRODUCTLINE_Kompakt'].unique()
print("Unique values in CONSTRUCTION_DESIGN_PREFAB_HOUSE:")
print(unique_values)

# Count NaN values in the specific column 'CONSTRUCTION_DESIGN_PREFAB_HOUSE'
nan_count = df_binary['PRODUCTLINE_Kompakt'].isna().sum()
print(f"Number of NaN values in 'CONSTRUCTION_DESIGN_PREFAB_HOUSE': {nan_count}")

Unique values in CONSTRUCTION_DESIGN_PREFAB_HOUSE:
[False  True]
Number of NaN values in 'CONSTRUCTION_DESIGN_PREFAB_HOUSE': 0


## Naive Bayes with expanding window

In [44]:
# Define parameter grids for GaussianNB and MultinomialNB
gnb_param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6]
}

mnb_param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0]
}

# Create a list of unique years in the data
years = sorted(contracts['YEAR'].unique())

# Initialize lists to store results
recall_scores = []
all_y_true = []
all_y_pred = []

# Loop over each year for expanding window estimation
for i in tqdm(range(len(years) - 1)):
    test_year = years[i + 1]
    train_years = years[:i + 1]  # All years up to and including the year before test_year

    # Split data into training and testing sets based on year
    train_indices = contracts['YEAR'].isin(train_years)
    test_indices = contracts['YEAR'] == test_year

    X_train = X[train_indices].reset_index(drop=True)
    y_train = y[train_indices].reset_index(drop=True)
    X_test = X[test_indices].reset_index(drop=True)
    y_test = y[test_indices].reset_index(drop=True)


    # Resample classes to ratio 1:5 in the training data
    train = pd.concat([X_train, y_train], axis=1)
    class_0 = train[train['DAMAGE'] == 0]
    class_1 = train[train['DAMAGE'] == 1]



    n_samples_0 = len(class_0) // 2  # Undersample class 0 to half its size
    n_samples_1 = n_samples_0 // 5   # Oversample class 1 to be 1/5 of the undersampled class 0

    # Resample class 0 (undersample)
    class_0_resampled = resample(class_0,
                                 replace=False,
                                 n_samples=n_samples_0,
                                 random_state=42)

    # Resample class 1 (oversample)
    class_1_resampled = resample(class_1,
                                 replace=True,
                                 n_samples=n_samples_1,
                                 random_state=42)

    # Combine the resampled classes
    train_resampled = pd.concat([class_0_resampled, class_1_resampled])
    X_train_resampled = train_resampled.drop(columns='DAMAGE')
    y_train_resampled = train_resampled['DAMAGE']

    # Train ComplementNB on discrete features
    cnb = ComplementNB()
    cnb.fit(X_train_resampled[continuous_features], y_train_resampled)

# Train BernoulliNB on binary features
    bnb = BernoulliNB()
    bnb.fit(X_train_resampled[discrete_features], y_train_resampled)

   # Get predicted probabilities for discrete features using ComplementNB
    cnb_prob = cnb.predict_proba(X_test[continuous_features])[:, 1]

# Get predicted probabilities for binary features using BernoulliNB
    bnb_prob = bnb.predict_proba(X_test[discrete_features])[:, 1]

    # Combine probabilities (you can average them, use weighted sum, etc.)
    combined_prob = (cnb_prob + bnb_prob) / 2

    # Convert probabilities to binary predictions based on a threshold (e.g., 0.5)
    y_pred = (combined_prob >= 0.5).astype(int)

    # Calculate accuracy for the current year
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate recall score
    recall = recall_score(y_test, y_pred)
    recall_scores.append((test_year, recall))

    # Print the classification report for each year
    print(f"Classification report for year {test_year}:")
    print(classification_report(y_test, y_pred))

    # Generating confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for year {test_year}:")
    print(conf_matrix)

    # Extracting TP, FP, TN, FN from the confusion matrix
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    TP = conf_matrix[1][1]

    # Printing TP, FP, TN, FN
    print("True Positives:", TP)
    print("False Positives:", FP)
    print("True Negatives:", TN)
    print("False Negatives:", FN)

    # Accumulate true and predicted values
    all_y_true.extend(y_test)
    all_y_pred.extend(y_pred)

    # Create a DataFrame for the probabilities
    probabilities_df = pd.DataFrame({'Probability': combined_prob})

    # Combine the probabilities with the original test data for the year
    combined_df = contracts[contracts['YEAR'] == test_year].copy()
    combined_df['Probability'] = probabilities_df['Probability'].values

    # Save the combined DataFrame to a CSV file
    combined_df.to_csv(f'probabilities_{test_year}.csv', index=False)

# Calculate and print the combined confusion matrix and classification report
combined_conf_matrix = confusion_matrix(all_y_true, all_y_pred)
print("Combined Confusion Matrix:")
print(combined_conf_matrix)

# Extract and print TP, FP, TN, FN from the combined confusion matrix
combined_TN = combined_conf_matrix[0][0]
combined_FP = combined_conf_matrix[0][1]
combined_FN = combined_conf_matrix[1][0]
combined_TP = combined_conf_matrix[1][1]

print("Combined True Positives:", combined_TP)
print("Combined False Positives:", combined_FP)
print("Combined True Negatives:", combined_TN)
print("Combined False Negatives:", combined_FN)

combined_class_report = classification_report(all_y_true, all_y_pred)
print("Combined Classification Report:")
print(combined_class_report)


# Combine all the CSV files into a single DataFrame
#combined_all_years = pd.concat([pd.read_csv(f'probabilities_{year}.csv') for year in years[1:]])

# Save the combined DataFrame to a CSV file
#combined_all_years.to_csv('/content/drive/MyDrive/Colab Notebooks/Capstone_Data/Capstone/Data/NaiveBayes.csv', index=False)
# Print recall scores
#for year, score in recall_scores:
    #print(f"Year: {year}, Recall Score: {score}")

  0%|          | 0/10 [00:00<?, ?it/s]

Classification report for year 2015:
              precision    recall  f1-score   support

           0       1.00      0.85      0.92    745209
           1       0.04      0.61      0.07      7391

    accuracy                           0.85    752600
   macro avg       0.52      0.73      0.50    752600
weighted avg       0.99      0.85      0.91    752600

Confusion Matrix for year 2015:
[[633981 111228]
 [  2885   4506]]
True Positives: 4506
False Positives: 111228
True Negatives: 633981
False Negatives: 2885


 10%|█         | 1/10 [00:13<02:03, 13.67s/it]

Classification report for year 2016:
              precision    recall  f1-score   support

           0       1.00      0.82      0.90    769344
           1       0.04      0.65      0.07      8184

    accuracy                           0.82    777528
   macro avg       0.52      0.73      0.48    777528
weighted avg       0.99      0.82      0.89    777528

Confusion Matrix for year 2016:
[[631241 138103]
 [  2878   5306]]
True Positives: 5306
False Positives: 138103
True Negatives: 631241
False Negatives: 2878


 20%|██        | 2/10 [00:29<01:58, 14.80s/it]

...

Classification report for year 2023:
              precision    recall  f1-score   support

           0       1.00      0.76      0.86    805948
           1       0.03      0.72      0.07      9672

    accuracy                           0.76    815620
   macro avg       0.52      0.74      0.46    815620
weighted avg       0.98      0.76      0.85    815620

Confusion Matrix for year 2023:
[[611884 194064]
 [  2691   6981]]
True Positives: 6981
False Positives: 194064
True Negatives: 611884
False Negatives: 2691


 90%|█████████ | 9/10 [03:09<00:24, 24.67s/it]

Classification report for year 2024:
              precision    recall  f1-score   support

           0       1.00      0.75      0.86    814414
           1       0.01      0.72      0.01      1866

    accuracy                           0.75    816280
   macro avg       0.50      0.74      0.44    816280
weighted avg       1.00      0.75      0.86    816280

Confusion Matrix for year 2024:
[[613634 200780]
 [   518   1348]]
True Positives: 1348
False Positives: 200780
True Negatives: 613634
False Negatives: 518


100%|██████████| 10/10 [03:41<00:00, 22.13s/it]


Combined Confusion Matrix:
[[6283173 1585942]
 [  30971   61364]]
Combined True Positives: 61364
Combined False Positives: 1585942
Combined True Negatives: 6283173
Combined False Negatives: 30971
Combined Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.80      0.89   7869115
           1       0.04      0.66      0.07     92335

    accuracy                           0.80   7961450
   macro avg       0.52      0.73      0.48   7961450
weighted avg       0.98      0.80      0.88   7961450

